In [35]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class EMNISTDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.X = df.iloc[:, 1:].values.reshape(-1, 28, 28).astype(np.uint8)
        self.y = df.iloc[:, 0].values.astype(np.int64) - 1  # oduzto 1 da bi bilo od0 do 25
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0) / 255.0
        label = torch.tensor(label, dtype=torch.long)
        return image, label

In [36]:
from torch.utils.data import DataLoader

train_dataset = EMNISTDataset("emnist-letters-train.csv")
test_dataset = EMNISTDataset("emnist-letters-test.csv")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)


In [37]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)  #28 na 14
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2) #14 na 7
        self.dropout = nn.Dropout(p=0.5)
        self.fcc1 = nn.Linear(64 * 7 * 7, 128)
        self.fcc2 = nn.Linear(128, 26)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1,64*7*7)
        x = F.relu(self.fcc1(x))
        x = self.dropout(x)
        x = self.fcc2(x)
        return x
    

    
    

In [ ]:
import os
import matplotlib.pyplot as plt
import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


num_epochs = 5
losses = []
accuracies = []

weights_dir = "weights-convolutional 1"

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    losses.append(total_loss)
    accuracies.append(val_acc)
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Val Acc: {val_acc:.2f}%")

   
    torch.save(model.state_dict(), os.path.join(weights_dir, f"model_epoch_{epoch+1}_{timestamp}.pth"))


plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(losses, marker='o')
plt.title('Loss po epohama')
plt.xlabel('Epohа')
plt.ylabel('Loss')
plt.subplot(1,2,2)
plt.plot(accuracies, marker='o', color='orange')
plt.title('Tačnost po epohama')
plt.xlabel('Epohа')
plt.ylabel('Tačnost (%)')
plt.tight_layout()
plt.show()

Epoch 1, Loss: 703.8073, Val Acc: 87.25%
Epoch 2, Loss: 378.4980, Val Acc: 88.99%
Epoch 2, Loss: 378.4980, Val Acc: 88.99%
